In [1]:
import cv2
import numpy as np
import dlib
import math

In [2]:
from drawFace import draw
import reference_world as world
from posepoints import head_pose_points

## Question 1

In [3]:
# To view the input image, if loaded properly or not
image = cv2.imread('test11.png')
# resized_image = cv2.resize(image,(720,540))
cv2.imshow('Original_image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of the input image
height = image.shape[0]
width = image.shape[1]
print('Image Height       : ',height)
print('Image Width        : ',width)

# To locate the faces within 80% of image dimensions
image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
# resized_image_roi = cv2.resize(image_roi, (720,540))
cv2.imshow('80_percent_ROI_Image',image_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of 80% of input image
height_roi = image_roi.shape[0]
width_roi = image_roi.shape[1]
print('80% of Total Image Height       : ',height_roi)
print('80% of Total Image Width        : ',width_roi)

# Convert the OpenCV image format (BGR) to Grayscale image
gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)

# Load Haar Cascade Classifier for the Frontal face and Profile Face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profileCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

# To perform Haar Cascade detection for the Frontal face and Profile Face detection
faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))
profiles = profileCascade.detectMultiScale(image=image, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))

print("Found {0} Faces!".format(len(faces)))
print("Found {0} Profiles!".format(len(profiles)))

# To draw Bounding boxes for the Frontal faces and Profile faces detected
for (x, y, w, h) in faces:
    cv2.rectangle(image_roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

for (x, y, w, h) in profiles:
    cv2.rectangle(image_roi, (x, y), (x + w, y + h), (255, 0, 0), 2)

# resized_faces_detected = cv2.resize(image_roi, (720,540))
cv2.imshow('Faces_detected_image',image_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image Height       :  374
Image Width        :  665
80% of Total Image Height       :  299
80% of Total Image Width        :  532
Found 3 Faces!
Found 0 Profiles!


## Question 2

In [4]:
# To view the input image, if loaded properly or not
image = cv2.imread('test9.png')
# resized_image = cv2.resize(image,(720,540))
cv2.imshow('Original_image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of the input image
height = image.shape[0]
width = image.shape[1]
print('Image Height       : ',height)
print('Image Width        : ',width)

# To locate the faces within 80% of image dimensions
image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
# resized_image_roi = cv2.resize(image_roi, (720,540))
cv2.imshow('80_percent_ROI_Image',image_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of 80% of input image
height_roi = image_roi.shape[0]
width_roi = image_roi.shape[1]
print('80% of Total Image Height       : ',height_roi)
print('80% of Total Image Width        : ',width_roi)

# To find the mid point of the 80% of input image
X,Y = (width_roi/2, height_roi/2)

# Convert the OpenCV image format (BGR) to Grayscale image
gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)

# Load Haar Cascade Classifier for the Frontal face and Profile Face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
profileCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

# To perform Haar Cascade detection for the Frontal face and Profile Face detection
faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))
profiles = profileCascade.detectMultiScale(image=image, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))

print("Found {0} Faces!".format(len(faces)))
print("Found {0} Profiles!".format(len(profiles)))

# To mark the face closest to the center of the image as the one under focus
list = []
for (x, y, w, h) in faces:
    euclidean_dists = np.sqrt((((((x+w)+x)/2)-X)**2) + (((((y+h)+y)/2)-Y)**2))
    list.append(euclidean_dists)

min_val = min(list)
min_idx = list.index(min_val)
faces_list = faces.tolist()

# To draw Bounding boxes for the Face closest to the center and the other faces
for i in range(len(faces_list)):
    if i == min_idx:
        cv2.rectangle(image_roi, (faces_list[min_idx][0], faces_list[min_idx][1]), 
                      ((faces_list[min_idx][2]+faces_list[min_idx][0]), (faces_list[min_idx][3]+faces_list[min_idx][1])),
                       (0, 255, 0), 2)
    else:
        cv2.rectangle(image_roi, (faces_list[i][0], faces_list[i][1]), 
                      ((faces_list[i][2]+faces_list[i][0]), (faces_list[i][3]+faces_list[i][1])),
                       (0, 0, 255), 2)
        
# resized_faces_detected = cv2.resize(image_roi, (720,540))
cv2.imshow('Faces_detected_image',image_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image Height       :  408
Image Width        :  612
80% of Total Image Height       :  327
80% of Total Image Width        :  489
Found 5 Faces!
Found 0 Profiles!


## Question 3

In [5]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Read the input image and view it
image = cv2.imread("test4.png")
cv2.imshow('Original_image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of the input image
height = image.shape[0]
width = image.shape[1]
print('Image Height       : ',height)
print('Image Width        : ',width)

# To locate the faces within 80% of image dimensions
image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
cv2.imshow('80_percent_ROI_Image',image_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of 80% of input image
height_roi = image_roi.shape[0]
width_roi = image_roi.shape[1]
print('80% of Total Image Height       : ',height_roi)
print('80% of Total Image Width        : ',width_roi)

# Convert the OpenCV image format (BGR) to Grayscale image
gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)

# Use detector to find landmarks
faces = detector(gray)
print("Found {0} Faces!".format(len(faces)))

# To mark the Facial Landmarks for the face in focus i.e., near the centre of the image
for face in faces:
    x1 = face.left() # left point
    y1 = face.top() # top point
    x2 = face.right() # right point
    y2 = face.bottom() # bottom point

    # Create landmark object
    landmarks = predictor(image=gray, box=face)

    # Loop through all the points
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y

        # Draw a circle
        cv2.circle(img=image_roi, center=(x, y), radius=2, color=(0, 255, 0), thickness=-1)

# Display the image
cv2.imshow('Facial_landmarked_image',image_roi)
cv2.waitKey(delay=0)
cv2.destroyAllWindows()

Image Height       :  768
Image Width        :  1024
80% of Total Image Height       :  615
80% of Total Image Width        :  819
Found 5 Faces!


In [6]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Read the input image and view it
image = cv2.imread("test4.png")
cv2.imshow('Original_image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of the input image
height = image.shape[0]
width = image.shape[1]
print('Image Height       : ',height)
print('Image Width        : ',width)

# To locate the faces within 80% of image dimensions
image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
cv2.imshow('80_percent_ROI_Image',image_roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Find the height and the width of 80% of input image
height_roi = image_roi.shape[0]
width_roi = image_roi.shape[1]
print('80% of Total Image Height       : ',height_roi)
print('80% of Total Image Width        : ',width_roi)

# To find the mid point of the 80% of input image
X,Y = (width_roi/2, height_roi/2)

# Convert the OpenCV image format (BGR) to Grayscale image
gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)

# Use detector to find landmarks
faces = detector(gray)
print("Found {0} Faces!".format(len(faces)))

# To mark the Facial Landmarks for the face in focus i.e., near the centre of the image
lists = []
for face in faces:
    x1 = face.left()     # left point
    y1 = face.top()      # top point
    x2 = face.right()    # right point
    y2 = face.bottom()   # bottom point
    
    e_dists = np.sqrt(((((x2+x1)/2)-X)**2) + ((((y2+y1)/2)-Y)**2))
    lists.append(e_dists)
       
minimum_val = min(lists)
minimum_idx = lists.index(minimum_val)

# To draw Facial Landmarks for the Face closest to the center of the image
for i in range(len(faces)):
    if i == minimum_idx:
        # Create landmark object
        landmarks = predictor(image=gray, box=faces[i])

        # Loop through all the points
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y

            # Draw a circle
            cv2.circle(img=image_roi, center=(x, y), radius=2, color=(0, 255, 0), thickness=-1)

# Display the image
cv2.imshow('Facial_landmarked_image',image_roi)
cv2.waitKey(delay=0)
cv2.destroyAllWindows()

Image Height       :  768
Image Width        :  1024
80% of Total Image Height       :  615
80% of Total Image Width        :  819
Found 5 Faces!


## Question 4

In [7]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    #Read the input image
    image = cv2.imread("test4.png")
    
    #Find the shape of the input image
    size_ori = image.shape
    height = image.shape[0]
    width = image.shape[1]
    
    # To locate the faces within 80% of image dimensions
    image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
    
    #Find the shape of the 80% of input image
    size_roi = image_roi.shape
    height_roi = image_roi.shape[0]
    width_roi = image_roi.shape[1]
    
    # Convert the OpenCV image format (BGR) to Grayscale image
    gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    print("Found {0} Faces!".format(len(faces)))

    # To get the 3D model points of the face
    face_3D_model = world.ref3DModel()

    # To find the Head Pose using the Landmarks 
    for face in faces:
        # Create landmark object
        landmarks = predictor(image=gray, box=face)
        
        # Draw polylines for detection of facial landmarks 
        draw(image_roi, landmarks)
        
        # To get the 2D image points of the six facial landmarks
        ref_Img_Pts = world.ref2dImagePoints(landmarks)
        
        # To find the focal length and the Camera Matrix
        focal_length = size_roi[1]
        camera_matrix = world.cameraMatrix(focal_length, (size_roi[0]/2, size_roi[1]/2))
        
        # Assuming no lens distortion
        dist_coeffs = np.zeros((4,1)) 

        # Calculate rotation and translation vector using solvePnP
        success, rotation_vector, translation_vector = cv2.solvePnP(face_3D_model, ref_Img_Pts, camera_matrix, dist_coeffs)
        
        # To project the 3D point onto the image plane
        nose_end_points_3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
        nose_end_point_2D, jacobian = cv2.projectPoints(nose_end_points_3D, rotation_vector, translation_vector, camera_matrix, 
                                                        dist_coeffs)

        # To find the angle Theta and Phi 
        p1 = (int(ref_Img_Pts[0, 0]), int(ref_Img_Pts[0, 1]))
        p2 = (int(nose_end_point_2D[0, 0, 0]), int(nose_end_point_2D[0, 0, 1]))
        x1, x2 = head_pose_points(image_roi, rotation_vector, translation_vector, camera_matrix)
        
        cv2.line(image_roi, p1, p2, (110, 220, 0),thickness=2, lineType=cv2.LINE_AA)
        cv2.line(image_roi, tuple(x1), tuple(x2), (255, 255, 0), 2)
        
        # To calculate the gaze angles (Yaw and Pitch)
        r_mat, jac = cv2.Rodrigues(rotation_vector)         # Converts a rotation matrix to a rotation vector or vice versa
        angles, mtx_R, mtx_Q, Qx, Qy, Qz = cv2.RQDecomp3x3(r_mat) # Computes an RQ decomposition of 3x3 matrices
        
        print('*' * 80)
        print("Angle: ", angles)

        x = np.arctan2(Qx[2][1], Qx[2][2])
        y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
        z = np.arctan2(Qz[0][0], Qz[1][0])
        print("AxisX: ", x)
        print("AxisY: ", y)
        print("AxisZ: ", z)
        print('*' * 80)

        gaze = "Looking: "
        if angles[1] < -15 and angles[0] < -25:
            gaze += "Left and Down"
        elif angles[1] > 15 and angles[0] > 25:
            gaze += "Right and Up"
        elif angles[1] > 15 and angles[0] < -25:
            gaze += "Right and Down"
        elif angles[1] < -15 and angles[0] > 25:
            gaze += "Left and Up"
        else:
            gaze += "Forward"
        
    cv2.putText(image_roi, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
    cv2.imshow("Head Pose", image_roi)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break

cv2.destroyAllWindows()

Found 5 Faces!
********************************************************************************
Angle:  (-151.14347135136384, -11.338632032218703, -4.831394199153405)
AxisX:  2.6379512179750226
AxisY:  0.19789646163431163
AxisZ:  1.4864725905579166
********************************************************************************
********************************************************************************
Angle:  (-167.8377311636898, -8.502576288958757, -0.1448681053995276)
AxisX:  2.9293210178834803
AxisY:  0.14839795114433044
AxisZ:  1.5682679013745375
********************************************************************************
********************************************************************************
Angle:  (-156.16224181165973, 22.648116337314654, 8.185333995146681)
AxisX:  2.725545286909017
AxisY:  -0.39528419946085913
AxisZ:  1.7136573553856274
********************************************************************************
*******************************************

In [8]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    #Read the input image
    image = cv2.imread("test4.png")
    
    #Find the shape of the input image
    size_ori = image.shape
    height = image.shape[0]
    width = image.shape[1]
    
    # To locate the faces within 80% of image dimensions
    image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
    
    #Find the shape of the 80% of input image
    size_roi = image_roi.shape
    height_roi = image_roi.shape[0]
    width_roi = image_roi.shape[1]
    
    X,Y = (width_roi/2, height_roi/2)
    
    # Convert the OpenCV image format (BGR) to Grayscale image
    gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    print("Found {0} Faces!".format(len(faces)))

    # To get the 3D model points of the face
    face_3D_model = world.ref3DModel()
    
    lists = []
    # To find the Head Pose using the Landmarks 
    for face in faces:
        x1 = face.left()     # left point
        y1 = face.top()      # top point
        x2 = face.right()    # right point
        y2 = face.bottom()   # bottom point

        e_dists = np.sqrt(((((x2+x1)/2)-X)**2) + ((((y2+y1)/2)-Y)**2))
        lists.append(e_dists)
        
    minimum_val = min(lists)
    minimum_idx = lists.index(minimum_val)
        
    for i in range(len(faces)):
        if i == minimum_idx:
            
            # Create landmark object
            landmarks = predictor(image=gray, box=faces[i])
        
            # Draw polylines for detection of facial landmarks 
            draw(image_roi, landmarks)

            # To get the 2D image points of the six facial landmarks
            ref_Img_Pts = world.ref2dImagePoints(landmarks)

            # To find the focal length and the Camera Matrix
            focal_length = size_roi[1]
            camera_matrix = world.cameraMatrix(focal_length, (size_roi[0]/2, size_roi[1]/2))

            # Assuming no lens distortion
            dist_coeffs = np.zeros((4,1)) 

            # Calculate rotation and translation vector using solvePnP
            success, rotation_vector, translation_vector = cv2.solvePnP(face_3D_model, ref_Img_Pts, camera_matrix, dist_coeffs)

            # To project the 3D point onto the image plane
            nose_end_points_3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
            nose_end_point_2D, jacobian = cv2.projectPoints(nose_end_points_3D, rotation_vector, translation_vector, camera_matrix, 
                                                            dist_coeffs)

            # To find the angle Theta and Phi 
            p1 = (int(ref_Img_Pts[0, 0]), int(ref_Img_Pts[0, 1]))
            p2 = (int(nose_end_point_2D[0, 0, 0]), int(nose_end_point_2D[0, 0, 1]))
            x1, x2 = head_pose_points(image_roi, rotation_vector, translation_vector, camera_matrix)

            cv2.line(image_roi, p1, p2, (110, 220, 0),thickness=2, lineType=cv2.LINE_AA)
            cv2.line(image_roi, tuple(x1), tuple(x2), (255, 255, 0), 2)

            # To calculate the gaze angles (Yaw and Pitch)
            r_mat, jac = cv2.Rodrigues(rotation_vector)         # Converts a rotation matrix to a rotation vector or vice versa
            angles, mtx_R, mtx_Q, Qx, Qy, Qz = cv2.RQDecomp3x3(r_mat) # Computes an RQ decomposition of 3x3 matrices

            print('*' * 80)
            print("Angle: ", angles)

            x = np.arctan2(Qx[2][1], Qx[2][2])
            y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
            z = np.arctan2(Qz[0][0], Qz[1][0])
            print("AxisX: ", x)
            print("AxisY: ", y)
            print("AxisZ: ", z)
            print('*' * 80)

            gaze = "Looking: "
            if angles[1] < -15 and angles[0] < -25:
                gaze += "Left and Down"
            elif angles[1] > 15 and angles[0] > 25:
                gaze += "Right and Up"
            elif angles[1] > 15 and angles[0] < -25:
                gaze += "Right and Down"
            elif angles[1] < -15 and angles[0] > 25:
                gaze += "Left and Up"
            else:
                gaze += "Forward"

    cv2.putText(image_roi, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
    cv2.imshow("Head Pose", image_roi)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break

cv2.destroyAllWindows()

Found 5 Faces!
********************************************************************************
Angle:  (-167.8377311636898, -8.502576288958757, -0.1448681053995276)
AxisX:  2.9293210178834803
AxisY:  0.14839795114433044
AxisZ:  1.5682679013745375
********************************************************************************
Found 5 Faces!
********************************************************************************
Angle:  (-167.8377311636898, -8.502576288958757, -0.1448681053995276)
AxisX:  2.9293210178834803
AxisY:  0.14839795114433044
AxisZ:  1.5682679013745375
********************************************************************************
Found 5 Faces!
********************************************************************************
Angle:  (-167.8377311636898, -8.502576288958757, -0.1448681053995276)
AxisX:  2.9293210178834803
AxisY:  0.14839795114433044
AxisZ:  1.5682679013745375
********************************************************************************
Found 5 Faces

## Question 5 v1

In [9]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Load the captured standalone video 
cap = cv2.VideoCapture("video.mp4")

while True:
    ret, image = cap.read()
    
    if not ret:
        print(f"[ERROR - System]Cannot read from source: {source}")
        break
    
    #Find the shape of the input image frame
    size_ori = image.shape
    height = image.shape[0]
    width = image.shape[1]
    
     # To locate the faces within 80% of image dimensions
    image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
    image_roi = cv2.resize(image_roi, (720,540))
    
    #Find the shape of the 80% of input image
    size_roi = image_roi.shape
    height_roi = image_roi.shape[0]
    width_roi = image_roi.shape[1]
        
    # Convert the OpenCV image format (BGR) to Grayscale image
    gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    print("Found {0} Faces!".format(len(faces)))
    
    # To get the 3D model points of the face
    face_3D_model = world.ref3DModel()

    # To find the Head Pose using the Landmarks 
    for face in faces:
        # Create landmark object
        landmarks = predictor(image=gray, box=face)
        
        # Draw polylines for detection of facial landmarks 
        draw(image_roi, landmarks)
        
        # To get the 2D image points of the six facial landmarks
        ref_Img_Pts = world.ref2dImagePoints(landmarks)
        
        # To find the focal length and the Camera Matrix
        focal_length = size_roi[1]
        camera_matrix = world.cameraMatrix(focal_length, (size_roi[0]/2, size_roi[1]/2))
        
        # Assuming no lens distortion
        dist_coeffs = np.zeros((4,1)) 

        # Calculate rotation and translation vector using solvePnP
        success, rotation_vector, translation_vector = cv2.solvePnP(face_3D_model, ref_Img_Pts, camera_matrix, dist_coeffs)
        
        # To project the 3D point onto the image plane
        nose_end_points_3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
        nose_end_point_2D, jacobian = cv2.projectPoints(nose_end_points_3D, rotation_vector, translation_vector, camera_matrix, 
                                                        dist_coeffs)

        # To find the angle Theta and Phi 
        p1 = (int(ref_Img_Pts[0, 0]), int(ref_Img_Pts[0, 1]))
        p2 = (int(nose_end_point_2D[0, 0, 0]), int(nose_end_point_2D[0, 0, 1]))
        x1, x2 = head_pose_points(image_roi, rotation_vector, translation_vector, camera_matrix)
        
        cv2.line(image_roi, p1, p2, (110, 220, 0),thickness=2, lineType=cv2.LINE_AA)
        cv2.line(image_roi, tuple(x1), tuple(x2), (255, 255, 0), 2)

        # To calculate the gaze angles (Yaw and Pitch)
        r_mat, jac = cv2.Rodrigues(rotation_vector)         # Converts a rotation matrix to a rotation vector or vice versa
        angles, mtx_R, mtx_Q, Qx, Qy, Qz = cv2.RQDecomp3x3(r_mat) # Computes an RQ decomposition of 3x3 matrices
        
        print('*' * 80)
        print("Angle: ", angles)
        
        x = np.arctan2(Qx[2][1], Qx[2][2])
        y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
        z = np.arctan2(Qz[0][0], Qz[1][0])
        print("AxisX: ", x)
        print("AxisY: ", y)
        print("AxisZ: ", z)
        print('*' * 80)

        gaze = "Looking: "
        if angles[1] < -15 and angles[0] < -25:
            gaze += "Left and Down"
        elif angles[1] > 15 and angles[0] > 25:
            gaze += "Right and Up"
        elif angles[1] > 15 and angles[0] < -25:
            gaze += "Right and Down"
        elif angles[1] < -15 and angles[0] > 25:
            gaze += "Left and Up"
        else:
            gaze += "Forward"
            
    cv2.putText(image_roi, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
    cv2.imshow("Head Pose", image_roi)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Found 3 Faces!
********************************************************************************
Angle:  (-118.55654620304688, 78.28794179037904, 42.864159715937284)
AxisX:  2.069202081035951
AxisY:  -1.3663823488517786
AxisZ:  2.3189170449387193
********************************************************************************
********************************************************************************
Angle:  (-158.70553906731917, 34.66219540882795, 4.298545828338325)
AxisX:  2.7699341978772103
AxisY:  -0.6049694358537094
AxisZ:  1.6458201045472658
********************************************************************************
********************************************************************************
Angle:  (-156.8773284669834, -58.16738023492119, -13.073394497239796)
AxisX:  2.73802590348149
AxisY:  1.0152123023588475
AxisZ:  1.3426225484059535
********************************************************************************
Found 3 Faces!
***********************************

In [10]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Load the captured standalone video 
cap = cv2.VideoCapture("video.mp4")

while True:
    ret, image = cap.read()
    
    if not ret:
        print(f"[ERROR - System]Cannot read from source: {source}")
        break
    
    #Find the shape of the input image frame
    size_ori = image.shape
    height = image.shape[0]
    width = image.shape[1]
    
     # To locate the faces within 80% of image dimensions
    image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
    image_roi = cv2.resize(image_roi, (720,540))
    
    #Find the shape of the 80% of input image
    size_roi = image_roi.shape
    height_roi = image_roi.shape[0]
    width_roi = image_roi.shape[1]
    
    X,Y = (width_roi/2, height_roi/2)
    
    # Convert the OpenCV image format (BGR) to Grayscale image
    gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    print("Found {0} Faces!".format(len(faces)))

    # To get the 3D model points of the face
    face_3D_model = world.ref3DModel()
    
    lists = []
    # To find the Head Pose using the Landmarks 
    for face in faces:
        x1 = face.left()     # left point
        y1 = face.top()      # top point
        x2 = face.right()    # right point
        y2 = face.bottom()   # bottom point

        e_dists = np.sqrt(((((x2+x1)/2)-X)**2) + ((((y2+y1)/2)-Y)**2))
        lists.append(e_dists)
        
    minimum_val = min(lists)
    minimum_idx = lists.index(minimum_val)
        
    for i in range(len(faces)):
        if i == minimum_idx:
            # Create landmark object
            landmarks = predictor(image=gray, box=faces[i])
        
            # Draw polylines for detection of facial landmarks 
            draw(image_roi, landmarks)

            # To get the 2D image points of the six facial landmarks
            ref_Img_Pts = world.ref2dImagePoints(landmarks)

            # To find the focal length and the Camera Matrix
            focal_length = size_roi[1]
            camera_matrix = world.cameraMatrix(focal_length, (size_roi[0]/2, size_roi[1]/2))

            # Assuming no lens distortion
            dist_coeffs = np.zeros((4,1)) 

            # Calculate rotation and translation vector using solvePnP
            success, rotation_vector, translation_vector = cv2.solvePnP(face_3D_model, ref_Img_Pts, camera_matrix, dist_coeffs)

            # To project the 3D point onto the image plane
            nose_end_points_3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
            nose_end_point_2D, jacobian = cv2.projectPoints(nose_end_points_3D, rotation_vector, translation_vector, camera_matrix, 
                                                            dist_coeffs)
            
            # To find the angle Theta and Phi 
            p1 = (int(ref_Img_Pts[0, 0]), int(ref_Img_Pts[0, 1]))
            p2 = (int(nose_end_point_2D[0, 0, 0]), int(nose_end_point_2D[0, 0, 1]))
            x1, x2 = head_pose_points(image_roi, rotation_vector, translation_vector, camera_matrix)

            cv2.line(image_roi, p1, p2, (110, 220, 0),thickness=2, lineType=cv2.LINE_AA)
            cv2.line(image_roi, tuple(x1), tuple(x2), (255, 255, 0), 2)

            # To calculate the gaze angles (Yaw and Pitch)
            r_mat, jac = cv2.Rodrigues(rotation_vector)         # Converts a rotation matrix to a rotation vector or vice versa
            angles, mtx_R, mtx_Q, Qx, Qy, Qz = cv2.RQDecomp3x3(r_mat) # Computes an RQ decomposition of 3x3 matrices

            print('*' * 80)
            print("Angle: ", angles)

            x = np.arctan2(Qx[2][1], Qx[2][2])
            y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
            z = np.arctan2(Qz[0][0], Qz[1][0])
            print("AxisX: ", x)
            print("AxisY: ", y)
            print("AxisZ: ", z)
            print('*' * 80)
            
            gaze = "Looking: "
            if angles[1] < -15 and angles[0] < -25:
                gaze += "Left and Down"
            elif angles[1] > 15 and angles[0] > 25:
                gaze += "Right and Up"
            elif angles[1] > 15 and angles[0] < -25:
                gaze += "Right and Down"
            elif angles[1] < -15 and angles[0] > 25:
                gaze += "Left and Up"
            else:
                gaze += "Forward"

    cv2.putText(image_roi, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
    cv2.imshow("Head Pose", image_roi)
    
    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Found 3 Faces!
********************************************************************************
Angle:  (-158.70553906731917, 34.66219540882795, 4.298545828338325)
AxisX:  2.7699341978772103
AxisY:  -0.6049694358537094
AxisZ:  1.6458201045472658
********************************************************************************
Found 3 Faces!
********************************************************************************
Angle:  (-158.87869330848852, 35.36713724342534, 4.537259583193529)
AxisX:  2.772956309499408
AxisY:  -0.6172729919024832
AxisZ:  1.6499864455392899
********************************************************************************
Found 3 Faces!
********************************************************************************
Angle:  (-159.24086601202382, 36.52835337259564, 4.556686652809901)
AxisX:  2.7792774156369484
AxisY:  -0.6375400366837687
AxisZ:  1.6503255118681097
********************************************************************************
Found 3 Faces!
*****

In [11]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Load the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, image = cap.read()
    
    if not ret:
        print(f"[ERROR - System]Cannot read from source: {source}")
        break
    
    #Find the shape of the input image frame
    size_ori = image.shape
    height = image.shape[0]
    width = image.shape[1]
    
     # To locate the faces within 80% of image dimensions
    image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
    image_roi = cv2.resize(image_roi, (720,540))
    
    #Find the shape of the 80% of input image
    size_roi = image_roi.shape
    height_roi = image_roi.shape[0]
    width_roi = image_roi.shape[1]
        
    # Convert the OpenCV image format (BGR) to Grayscale image
    gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    print("Found {0} Faces!".format(len(faces)))
    
    # To get the 3D model points of the face
    face_3D_model = world.ref3DModel()

    # To find the Head Pose using the Landmarks 
    for face in faces:
        # Create landmark object
        landmarks = predictor(image=gray, box=face)
        
        # Draw polylines for detection of facial landmarks 
        draw(image_roi, landmarks)
        
        # To get the 2D image points of the six facial landmarks
        ref_Img_Pts = world.ref2dImagePoints(landmarks)
        
        # To find the focal length and the Camera Matrix
        focal_length = size_roi[1]
        camera_matrix = world.cameraMatrix(focal_length, (size_roi[0]/2, size_roi[1]/2))
        
        # Assuming no lens distortion
        dist_coeffs = np.zeros((4,1)) 

        # Calculate rotation and translation vector using solvePnP
        success, rotation_vector, translation_vector = cv2.solvePnP(face_3D_model, ref_Img_Pts, camera_matrix, dist_coeffs)
        
        # To project the 3D point onto the image plane
        nose_end_points_3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
        nose_end_point_2D, jacobian = cv2.projectPoints(nose_end_points_3D, rotation_vector, translation_vector, camera_matrix, 
                                                        dist_coeffs)

        # To find the angle Theta and Phi 
        p1 = (int(ref_Img_Pts[0, 0]), int(ref_Img_Pts[0, 1]))
        p2 = (int(nose_end_point_2D[0, 0, 0]), int(nose_end_point_2D[0, 0, 1]))
        x1, x2 = head_pose_points(image_roi, rotation_vector, translation_vector, camera_matrix)
        
        cv2.line(image_roi, p1, p2, (110, 220, 0),thickness=2, lineType=cv2.LINE_AA)
        cv2.line(image_roi, tuple(x1), tuple(x2), (255, 255, 0), 2)

        # To calculate the gaze angles (Yaw and Pitch)
        r_mat, jac = cv2.Rodrigues(rotation_vector)         # Converts a rotation matrix to a rotation vector or vice versa
        angles, mtx_R, mtx_Q, Qx, Qy, Qz = cv2.RQDecomp3x3(r_mat) # Computes an RQ decomposition of 3x3 matrices
        
        print('*' * 80)
        print("Angle: ", angles)
        
        x = np.arctan2(Qx[2][1], Qx[2][2])
        y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
        z = np.arctan2(Qz[0][0], Qz[1][0])
        print("AxisX: ", x)
        print("AxisY: ", y)
        print("AxisZ: ", z)
        print('*' * 80)

        gaze = "Looking: "
        if angles[1] < -15 and angles[0] < -25:
            gaze += "Left and Down"
        elif angles[1] > 15 and angles[0] > 25:
            gaze += "Right and Up"
        elif angles[1] > 15 and angles[0] < -25:
            gaze += "Right and Down"
        elif angles[1] < -15 and angles[0] > 25:
            gaze += "Left and Up"
        else:
            gaze += "Forward"
            
    cv2.putText(image_roi, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
    cv2.imshow("Head Pose", image_roi)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 1 Faces!
********************************************************************************
Angle:  (179.61138158538213, -9.18540254228512, -5.553935959044998)
AxisX:  -3.134809982720856
AxisY:  0.16031551748393233
AxisZ:  1.473861857864653
********************************************************************************
Found 1 Faces!
********************************************************************************
Angle:  (177.33699710487426, -8.858113989379198, -5.047828143364709)
AxisX:  -3.0951144850797085
AxisY:  0.1546032546316368
AxisZ:  1.4826951056183502
********************************************************************************
Found 1 Faces!
********************************************************************************
Angle:  (177.33699710487426, -8.858113989379198, -5.047828143364709)
AxisX:  -3.0951144850797085
AxisY:  0.1546032546316368
AxisZ:  1.4826951056183502
***************************************

In [12]:
# Load the detector
detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Load the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, image = cap.read()
    
    if not ret:
        print(f"[ERROR - System]Cannot read from source: {source}")
        break
    
    #Find the shape of the input image frame
    size_ori = image.shape
    height = image.shape[0]
    width = image.shape[1]
    
     # To locate the faces within 80% of image dimensions
    image_roi = image[int(0.1*height):int(0.9*height),int(0.1*width):int(0.9*width)]
    image_roi = cv2.resize(image_roi, (720,540))
    
    #Find the shape of the 80% of input image
    size_roi = image_roi.shape
    height_roi = image_roi.shape[0]
    width_roi = image_roi.shape[1]
    
    X,Y = (width_roi/2, height_roi/2)
    
    # Convert the OpenCV image format (BGR) to Grayscale image
    gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    
    # Use detector to find landmarks
    faces = detector(gray)
    print("Found {0} Faces!".format(len(faces)))

    # To get the 3D model points of the face
    face_3D_model = world.ref3DModel()
    
    lists = []
    # To find the Head Pose using the Landmarks 
    for face in faces:
        x1 = face.left()     # left point
        y1 = face.top()      # top point
        x2 = face.right()    # right point
        y2 = face.bottom()   # bottom point

        e_dists = np.sqrt(((((x2+x1)/2)-X)**2) + ((((y2+y1)/2)-Y)**2))
        lists.append(e_dists)
        
    minimum_val = min(lists)
    minimum_idx = lists.index(minimum_val)
        
    for i in range(len(faces)):
        if i == minimum_idx:
            # Create landmark object
            landmarks = predictor(image=gray, box=faces[i])
        
            # Draw polylines for detection of facial landmarks 
            draw(image_roi, landmarks)

            # To get the 2D image points of the six facial landmarks
            ref_Img_Pts = world.ref2dImagePoints(landmarks)

            # To find the focal length and the Camera Matrix
            focal_length = size_roi[1]
            camera_matrix = world.cameraMatrix(focal_length, (size_roi[0]/2, size_roi[1]/2))

            # Assuming no lens distortion
            dist_coeffs = np.zeros((4,1)) 

            # Calculate rotation and translation vector using solvePnP
            success, rotation_vector, translation_vector = cv2.solvePnP(face_3D_model, ref_Img_Pts, camera_matrix, dist_coeffs)

            # To project the 3D point onto the image plane
            nose_end_points_3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
            nose_end_point_2D, jacobian = cv2.projectPoints(nose_end_points_3D, rotation_vector, translation_vector, camera_matrix, 
                                                            dist_coeffs)
            
            # To find the angle Theta and Phi 
            p1 = (int(ref_Img_Pts[0, 0]), int(ref_Img_Pts[0, 1]))
            p2 = (int(nose_end_point_2D[0, 0, 0]), int(nose_end_point_2D[0, 0, 1]))
            x1, x2 = head_pose_points(image_roi, rotation_vector, translation_vector, camera_matrix)

            cv2.line(image_roi, p1, p2, (110, 220, 0),thickness=2, lineType=cv2.LINE_AA)
            cv2.line(image_roi, tuple(x1), tuple(x2), (255, 255, 0), 2)

            # To calculate the gaze angles (Yaw and Pitch)
            r_mat, jac = cv2.Rodrigues(rotation_vector)         # Converts a rotation matrix to a rotation vector or vice versa
            angles, mtx_R, mtx_Q, Qx, Qy, Qz = cv2.RQDecomp3x3(r_mat) # Computes an RQ decomposition of 3x3 matrices

            print('*' * 80)
            print("Angle: ", angles)

            x = np.arctan2(Qx[2][1], Qx[2][2])
            y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
            z = np.arctan2(Qz[0][0], Qz[1][0])
            print("AxisX: ", x)
            print("AxisY: ", y)
            print("AxisZ: ", z)
            print('*' * 80)
            
            gaze = "Looking: "
            if angles[1] < -15 and angles[0] < -25:
                gaze += "Left and Down"
            elif angles[1] > 15 and angles[0] > 25:
                gaze += "Right and Up"
            elif angles[1] > 15 and angles[0] < -25:
                gaze += "Right and Down"
            elif angles[1] < -15 and angles[0] > 25:
                gaze += "Left and Up"
            else:
                gaze += "Forward"

    cv2.putText(image_roi, gaze, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 80), 2)
    cv2.imshow("Head Pose", image_roi)
    
    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Found 1 Faces!
********************************************************************************
Angle:  (173.8645817738546, -1.70358137741025, -2.428398511659084)
AxisX:  -3.0345094045566867
AxisY:  0.029733104111467505
AxisZ:  1.528412777215916
********************************************************************************
Found 1 Faces!
********************************************************************************
Angle:  (175.13219615722863, -1.977375949914388, -1.199264238806621)
AxisX:  -3.056633449192201
AxisY:  0.034511720875756484
AxisZ:  1.5498652172262997
********************************************************************************
Found 1 Faces!
********************************************************************************
Angle:  (174.20176619205648, -3.2665194841443235, -1.392657450397974)
AxisX:  -3.0403943828407325
AxisY:  0.057011520078866534
AxisZ:  1.5464898689330193
********************************************************************************
Found 1 Faces

Found 1 Faces!
********************************************************************************
Angle:  (176.48691248639219, -4.876314251930011, -3.105018317799356)
AxisX:  -3.0802777095666376
AxisY:  0.08510773905810202
AxisZ:  1.5166035338145614
********************************************************************************
Found 1 Faces!
********************************************************************************
Angle:  (177.13109094474137, -5.703270214751309, -3.8272178113118245)
AxisX:  -3.091520744635251
AxisY:  0.09954084337833477
AxisZ:  1.5039987747965349
********************************************************************************
Found 1 Faces!
********************************************************************************
Angle:  (176.9126522855396, -6.588941784556308, -2.8379128919378744)
AxisX:  -3.0877082708185393
AxisY:  0.11499872836273885
AxisZ:  1.5212654029457884
********************************************************************************
Found 1 Faces

## Question 5 v2

In [13]:
import cv2
import numpy as np
import dlib
import math

from drawFace import draw
import reference_world as world
from posepoints import head_pose_points

from face_detector import get_face_detector, find_faces
from face_landmarks import get_landmark_model, detect_marks

In [14]:
# Load the Detector model
face_model = get_face_detector()

# Load the Landmark model
landmark_model = get_landmark_model()

# Load the webcam
cap = cv2.VideoCapture(0)
ret, image = cap.read()

# Find the sahpe of the input image frame from the video
size = image.shape
font = cv2.FONT_HERSHEY_SIMPLEX 

# To get the 3D model points of the face
face_3D_model = world.ref3DModel()

# To find the focal length and the Camera Matrix
focal_length = size[1]
center = (size[1]/2, size[0]/2)
camera_matrix = world.cameraMatrix(focal_length, center)

while True:
    ret, image = cap.read()
    if ret == True:
        # To find the coordinates of the faces detected in an image
        faces = find_faces(image, face_model)
        for face in faces:
            # To find the Facial Landmarks of the detected faces in an image
            landmarks = detect_marks(image, landmark_model, face)
            
            # To get the 2D image points of the six facial landmarks
            image_points = np.array([
                                    landmarks[30],     # Nose tip
                                    landmarks[8],     # Chin
                                    landmarks[36],     # Left eye left corner
                                    landmarks[45],     # Right eye right corne
                                    landmarks[48],     # Left Mouth corner
                                    landmarks[54]      # Right mouth corner
                                ], dtype="double")
            
            # Assuming no lens distortion
            dist_coeffs = np.zeros((4,1)) 
            
            # Calculate rotation and translation vector using solvePnP
            (success, rotation_vector, translation_vector) = cv2.solvePnP(face_3D_model, image_points, camera_matrix,
                                                                          dist_coeffs, flags=cv2.SOLVEPNP_UPNP)
            
            # To project the 3D point onto the image plane
            (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector,
                                                             translation_vector, camera_matrix, dist_coeffs)
            
            # To find the angle Theta and Phi 
            for p in image_points:
                cv2.circle(image, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
                
            p1 = ( int(image_points[0][0]), int(image_points[0][1]))
            p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
            x1, x2 = head_pose_points(image, rotation_vector, translation_vector, camera_matrix)

            cv2.line(image, p1, p2, (0, 255, 255), 2)
            cv2.line(image, tuple(x1), tuple(x2), (255, 255, 0), 2)
            
            # For angle phi
            try:
                m = (p2[1] - p1[1])/(p2[0] - p1[0])
                ang1 = int(math.degrees(math.atan(m)))
            except:
                ang1 = 90
            
            # For angle theta
            try:
                m = (x2[1] - x1[1])/(x2[0] - x1[0])
                ang2 = int(math.degrees(math.atan(-1/m)))
            except:
                ang2 = 90
                
            if ang1 >= 48:
                print('Head down')
                cv2.putText(image, 'Head down', (30, 30), font, 2, (255, 255, 128), 3)
            elif ang1 <= -48:
                print('Head up')
                cv2.putText(image, 'Head up', (30, 30), font, 2, (255, 255, 128), 3)
             
            if ang2 >= 48:
                print('Head right')
                cv2.putText(image, 'Head right', (90, 30), font, 2, (255, 255, 128), 3)
            elif ang2 <= -48:
                print('Head left')
                cv2.putText(image, 'Head left', (90, 30), font, 2, (255, 255, 128), 3)
            
            cv2.putText(image, str(ang1), tuple(p1), font, 2, (128, 255, 255), 3)
            cv2.putText(image, str(ang2), tuple(x1), font, 2, (255, 255, 128), 3)
        cv2.imshow('Head_pose_webcam', image)
        
        key = cv2.waitKey(10) & 0xFF
        if key == 27:
            break
            
cv2.destroyAllWindows()
cap.release()